### Numerical example
### Consider the following problem over $\Omega = {(-1, 1)}^d$: 
### $$-\Delta u + \pi^2 u = f,$$
### with periodic condition
### $$ u(x_1 + p_1,\cdots,x_{k} + p_k,\cdots,x_d + p_d) = u(x_1,\cdots,x_{k},\cdots,x_d) $$ 
### Assume $u(x) = \sum_{i = 1}^d \cos(\pi x_i) + \cos(2 \pi x_i) $, we can get $f(x)$ and $p_1 = \cdots = p_d = 2$.
### Network structure
### construct a transform $x^{\prime} = \text{transform} (x)$ before the first fully connected layer of our neural network
### $$x = (x_1,\cdots,x_d) \in R^d \Rightarrow x^{\prime} \in R^{2d}$$
### where $x^{\prime}_{2i - 1} = \sin(2\pi x_i / p_i)$ and $x^{\prime}_{2i} = \cos(2\pi x_i / p_i)$ for $i = 1, 2, \cdots, d$.

In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR, MultiStepLR
import numpy as np
import matplotlib.pyplot as plt
from math import *
import time

In [2]:
torch.set_default_tensor_type('torch.DoubleTensor')

In [3]:
# activation function
def activation(x):
    return x * torch.sigmoid(x) 

In [4]:
def transform(x):
    x_transform = torch.zeros(len(x), 2*d)
    for index in range(d):
        x_transform[:, 2*index] = torch.sin(2*pi*x[:, index] / 2)
        x_transform[:, 2*index+1] = torch.cos(2*pi*x[:, index] / 2)
    return x_transform

In [5]:
# exact solution
def u_ex(x):  
#     x_temp_1 = torch.cos(pi*x)
#     u_temp_1 = (x_temp_1.sum(1)).reshape([x.size()[0], 1]) # x_temp.sum(1) # summation by row for x_temp
#     x_temp_2 = torch.cos(2*pi*x)
#     u_temp_2 = (x_temp_2.sum(1)).reshape([x.size()[0], 1]) 
#     x_temp_3 = torch.cos(4*pi*x)
#     u_temp_3 = (x_temp_3.sum(1)).reshape([x.size()[0], 1]) 
#     x_temp_4 = torch.cos(8*pi*x)
#     u_temp_4 = (x_temp_4.sum(1)).reshape([x.size()[0], 1]) 
#     u_temp = u_temp_1 + u_temp_2 + u_temp_3 + u_temp_4
    
    x_temp_1 = torch.cos(pi*x)
    u_temp_1 = (x_temp_1.sum(1)).reshape([x.size()[0], 1]) # x_temp.sum(1) # summation by row for x_temp
    x_temp_2 = torch.cos(2*pi*x)
    u_temp_2 = (x_temp_2.sum(1)).reshape([x.size()[0], 1]) 
   
    u_temp = u_temp_1 + u_temp_2 
    return u_temp

In [6]:
def f(x):
#     x_temp_1 = torch.cos(pi*x)
#     f_temp_1 = pi**2 * (x_temp_1.sum(1)).reshape([x.size()[0], 1]) 
#     x_temp_2 = torch.cos(2*pi*x)
#     f_temp_2 = (2*pi)**2 * (x_temp_2.sum(1)).reshape([x.size()[0], 1]) 
#     x_temp_3 = torch.cos(4*pi*x)
#     f_temp_3 = (4*pi)**2 * (x_temp_3.sum(1)).reshape([x.size()[0], 1]) 
#     x_temp_4 = torch.cos(8*pi*x)
#     f_temp_4 = (8*pi)**2 * (x_temp_4.sum(1)).reshape([x.size()[0], 1]) 
#     f_temp = f_temp_1 + f_temp_2 + f_temp_3 + f_temp_4 + pi**2 * u_ex(x)
    
    x_temp_1 = torch.cos(pi*x)
    f_temp_1 = pi**2 * (x_temp_1.sum(1)).reshape([x.size()[0], 1]) 
    x_temp_2 = torch.cos(2*pi*x)
    f_temp_2 = (2*pi)**2 * (x_temp_2.sum(1)).reshape([x.size()[0], 1]) 
    f_temp = f_temp_1 + f_temp_2 + pi**2 * u_ex(x)
    return f_temp

In [7]:
# build ResNet with three blocks
class Net(nn.Module):
    def __init__(self,input_size,width,output_size):
        super(Net,self).__init__()
        self.layer_in = nn.Linear(input_size,width)
        self.layer_1 = nn.Linear(width,width)
        self.layer_2 = nn.Linear(width,width)
        self.layer_3 = nn.Linear(width,width)
        self.layer_4 = nn.Linear(width,width)
        self.layer_5 = nn.Linear(width,width)
        self.layer_6 = nn.Linear(width,width)
        self.layer_out = nn.Linear(width,output_size)
    def forward(self,x):
        output = self.layer_in(transform(x)) # transform for periodic
        output = output + activation(self.layer_2(activation(self.layer_1(output)))) # residual block 1
        output = output + activation(self.layer_4(activation(self.layer_3(output)))) # residual block 2
        output = output + activation(self.layer_6(activation(self.layer_5(output)))) # residual block 3
        output = self.layer_out(output)
        return output

In [8]:
d = 2 # dimension of input
input_size = d * 2
width_1 = 8
width_2 = 8
output_size_1 = 1
output_size_2 = d
data_size = 1000

In [9]:
CUDA = torch.cuda.is_available()
# print('CUDA is: ', CUDA)
if CUDA:
    net_1 = Net(input_size, width_1, output_size_1).cuda() # network for u on gpu
    net_2 = Net(input_size, width_2, output_size_2).cuda() # network for grad u on gpu
else:
    net_1 = Net(input_size, width_1, output_size_1) # network for u on cpu
    net_2 = Net(input_size, width_2, output_size_2) # network for grad u on cpu

In [10]:
def generate_sample(data_size_temp):
    sample_temp = 2.0 * torch.rand(data_size_temp, d) - 1.0
    return sample_temp

In [11]:
def relative_l2_error():
    data_size_temp = 500
    x = generate_sample(data_size_temp) # cuda
    predict = net_1(x)
    exact = u_ex(x)
    value = torch.sqrt(torch.sum((predict - exact)**2))/torch.sqrt(torch.sum((exact)**2))
    return value

In [12]:
# Xavier normal initialization for weights:
#             mean = 0 std = gain * sqrt(2 / fan_in + fan_out)
# zero initialization for biases
def initialize_weights(self):
    for m in self.modules():
        if isinstance(m,nn.Linear):
            nn.init.xavier_normal_(m.weight.data)
            if m.bias is not None:
                m.bias.data.zero_()

In [13]:
initialize_weights(net_1)
initialize_weights(net_2)

In [14]:
# number of net_1 and net_2
param_num_1 = sum(x.numel() for x in net_1.parameters())
param_num_2 = sum(x.numel() for x in net_2.parameters())
# print(param_num_1)
# print(param_num_2)

In [15]:
def loss_function(x):
#     x = generate_sample(data_size)
#     x.requires_grad = True
    u_hat = net_1(x)
    grad_u_hat = torch.autograd.grad(outputs = u_hat, inputs = x, grad_outputs = torch.ones(u_hat.shape), create_graph = True)
    p_hat = net_2(x)
    part_1 = torch.sum((grad_u_hat[0] - p_hat)**2) / len(x)
    laplace_u = torch.zeros([len(p_hat), 1])
    for index in range(d):
        p_temp = p_hat[:, index].reshape([len(p_hat), 1])
        temp = torch.autograd.grad(outputs = p_temp, inputs = x, grad_outputs = torch.ones(p_temp.shape), create_graph = True, allow_unused = True)[0]
        laplace_u = temp[:, index].reshape([len(p_hat), 1]) + laplace_u
    part_2 = torch.sum((-laplace_u + pi**2 * u_hat - f(x))**2)  / len(x)
    return part_1 + part_2 

In [16]:
optimizer = optim.Adam([
                {'params': net_1.parameters()},
                {'params': net_2.parameters()},
            ])

In [17]:
epoch = 20000
loss_record = np.zeros(epoch)
error_record = np.zeros(epoch)
time_start = time.time()
for i in range(epoch):
    optimizer.zero_grad()
    x = generate_sample(data_size) # cuda
    x.requires_grad = True
    loss = loss_function(x)
    loss_record[i] = float(loss)
    error = relative_l2_error()
    error_record[i] = float(error)
    if i % 50 == 0:
        print("current epoch is: ", i)
        print("current loss is: ", loss.detach())
        print("current error is: ", error.detach())
    loss.backward()
    optimizer.step() 
time_end = time.time()
print('total time is: ', time_end-time_start, 'seconds')

current epoch is:  0
current loss is:  tensor(2514.0347)
current error is:  tensor(0.7671)
current epoch is:  50
current loss is:  tensor(2269.8381)
current error is:  tensor(0.8121)
current epoch is:  100
current loss is:  tensor(1506.9640)
current error is:  tensor(1.0614)
current epoch is:  150
current loss is:  tensor(768.5473)
current error is:  tensor(1.1566)
current epoch is:  200
current loss is:  tensor(287.2739)
current error is:  tensor(0.8885)
current epoch is:  250
current loss is:  tensor(141.5396)
current error is:  tensor(0.6352)
current epoch is:  300
current loss is:  tensor(67.9022)
current error is:  tensor(0.4318)
current epoch is:  350
current loss is:  tensor(37.3923)
current error is:  tensor(0.3133)
current epoch is:  400
current loss is:  tensor(25.5020)
current error is:  tensor(0.2301)
current epoch is:  450
current loss is:  tensor(19.0528)
current error is:  tensor(0.1934)
current epoch is:  500
current loss is:  tensor(15.2866)
current error is:  tensor(0

current epoch is:  4550
current loss is:  tensor(0.1720)
current error is:  tensor(0.0107)
current epoch is:  4600
current loss is:  tensor(0.1649)
current error is:  tensor(0.0106)
current epoch is:  4650
current loss is:  tensor(0.1612)
current error is:  tensor(0.0117)
current epoch is:  4700
current loss is:  tensor(0.1546)
current error is:  tensor(0.0108)
current epoch is:  4750
current loss is:  tensor(0.1591)
current error is:  tensor(0.0112)
current epoch is:  4800
current loss is:  tensor(0.1492)
current error is:  tensor(0.0102)
current epoch is:  4850
current loss is:  tensor(0.1448)
current error is:  tensor(0.0099)
current epoch is:  4900
current loss is:  tensor(0.1423)
current error is:  tensor(0.0105)
current epoch is:  4950
current loss is:  tensor(0.1349)
current error is:  tensor(0.0094)
current epoch is:  5000
current loss is:  tensor(0.1390)
current error is:  tensor(0.0094)
current epoch is:  5050
current loss is:  tensor(0.1329)
current error is:  tensor(0.0101)

current epoch is:  9100
current loss is:  tensor(0.0267)
current error is:  tensor(0.0037)
current epoch is:  9150
current loss is:  tensor(0.0271)
current error is:  tensor(0.0041)
current epoch is:  9200
current loss is:  tensor(0.0232)
current error is:  tensor(0.0030)
current epoch is:  9250
current loss is:  tensor(0.0233)
current error is:  tensor(0.0036)
current epoch is:  9300
current loss is:  tensor(0.0239)
current error is:  tensor(0.0034)
current epoch is:  9350
current loss is:  tensor(0.0245)
current error is:  tensor(0.0038)
current epoch is:  9400
current loss is:  tensor(0.0353)
current error is:  tensor(0.0050)
current epoch is:  9450
current loss is:  tensor(0.0294)
current error is:  tensor(0.0035)
current epoch is:  9500
current loss is:  tensor(0.0233)
current error is:  tensor(0.0032)
current epoch is:  9550
current loss is:  tensor(0.0509)
current error is:  tensor(0.0089)
current epoch is:  9600
current loss is:  tensor(0.0237)
current error is:  tensor(0.0041)

current epoch is:  13600
current loss is:  tensor(0.0121)
current error is:  tensor(0.0024)
current epoch is:  13650
current loss is:  tensor(0.0117)
current error is:  tensor(0.0024)
current epoch is:  13700
current loss is:  tensor(0.0124)
current error is:  tensor(0.0024)
current epoch is:  13750
current loss is:  tensor(0.0124)
current error is:  tensor(0.0032)
current epoch is:  13800
current loss is:  tensor(0.0164)
current error is:  tensor(0.0026)
current epoch is:  13850
current loss is:  tensor(0.0122)
current error is:  tensor(0.0020)
current epoch is:  13900
current loss is:  tensor(0.0146)
current error is:  tensor(0.0024)
current epoch is:  13950
current loss is:  tensor(0.0125)
current error is:  tensor(0.0024)
current epoch is:  14000
current loss is:  tensor(0.0323)
current error is:  tensor(0.0066)
current epoch is:  14050
current loss is:  tensor(0.0103)
current error is:  tensor(0.0020)
current epoch is:  14100
current loss is:  tensor(0.0149)
current error is:  ten

current epoch is:  18100
current loss is:  tensor(0.0090)
current error is:  tensor(0.0032)
current epoch is:  18150
current loss is:  tensor(0.0076)
current error is:  tensor(0.0016)
current epoch is:  18200
current loss is:  tensor(0.0076)
current error is:  tensor(0.0021)
current epoch is:  18250
current loss is:  tensor(0.0079)
current error is:  tensor(0.0017)
current epoch is:  18300
current loss is:  tensor(0.0088)
current error is:  tensor(0.0021)
current epoch is:  18350
current loss is:  tensor(0.0088)
current error is:  tensor(0.0020)
current epoch is:  18400
current loss is:  tensor(0.0082)
current error is:  tensor(0.0022)
current epoch is:  18450
current loss is:  tensor(0.0075)
current error is:  tensor(0.0015)
current epoch is:  18500
current loss is:  tensor(0.0182)
current error is:  tensor(0.0039)
current epoch is:  18550
current loss is:  tensor(0.0174)
current error is:  tensor(0.0057)
current epoch is:  18600
current loss is:  tensor(0.0165)
current error is:  ten